In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
import dask.dataframe as ddf
import dask

import tensorflow as tf

### Load NYC Taxi fare prepped data

In [2]:
train_df = (pd
            .read_parquet("../../datasets/kaggle/new-york-city-taxi-fare-prediction/train_full_2")
            .sample(frac=0.80, random_state=42)
           )

In [3]:
print(train_df.shape)
print(train_df.columns)
print(train_df.info())

(43397466, 46)
Index(['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'distance_miles', 'is_pickup_JFK_new', 'is_dropoff_JFK_new',
       'is_pickup_EWR_new', 'is_dropoff_EWR_new', 'is_pickup_LGA_new',
       'is_dropoff_LGA_new', 'is_to_from_JFK_new', 'distance_to_center',
       'hour', 'weekday', 'year_2009', 'year_2010', 'year_2011', 'year_2012',
       'year_2013', 'year_2014', 'year_2015', 'hour_period_4', 'hour_period_0',
       'hour_period_1', 'hour_period_2', 'hour_period_5', 'hour_period_3',
       'distance_to_center_dropoff', 'pickup_zone_2', 'pickup_zone_0',
       'pickup_zone_1', 'pickup_zone_3', 'pickup_zone_4', 'pickup_zone_5',
       'pickup_zone_6', 'dropoff_zone_2', 'dropoff_zone_0', 'dropoff_zone_1',
       'dropoff_zone_3', 'dropoff_zone_4', 'dropoff_zone_5', 'dropoff_zone_6'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 43397466 entries,

In [4]:
cols = [
    "distance_miles",
    "passenger_count",
    "is_pickup_JFK_new",
    "is_dropoff_JFK_new",
    #"is_pickup_EWR_new",
    "is_dropoff_EWR_new",
    #"is_pickup_LGA_new",
    #"is_dropoff_LGA_new",
    #"is_to_from_JFK_new",
    "distance_to_center",
    "distance_to_center_dropoff",
    "year_2009",
    "year_2010",
    "year_2011",
    "year_2012",
    "year_2013",
    "year_2014",
    "year_2015",
    #"hour_period_0",
    #"hour_period_1",
    #"hour_period_2",
    #"hour_period_3",
    "hour_period_4",
    #"hour_period_5",
    "pickup_zone_0",
    "pickup_zone_1",
    "pickup_zone_2",
    "pickup_zone_3",
    "pickup_zone_4",
    "pickup_zone_5",
    "pickup_zone_6",
    "dropoff_zone_0",
    "dropoff_zone_1",
    "dropoff_zone_2",
    "dropoff_zone_3",
    "dropoff_zone_4",
    "dropoff_zone_5",
    "dropoff_zone_6",
]
x_train = train_df[cols].values
y_train = train_df[['fare_amount']].values

In [5]:
import gc
del train_df
gc.collect()

28

In [6]:
#scaler = preprocessing.StandardScaler()
scaler = preprocessing.MinMaxScaler()
x_train_norm = scaler.fit_transform(x_train)

/home/subhachandra/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype object was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [7]:
del x_train
gc.collect()

0

In [8]:
feature_columns = [
    tf.feature_column.numeric_column('x', shape=np.array(x_train_norm).shape[1:])]

In [9]:
from datetime import datetime
print(datetime.now())
global_step = tf.Variable(0, trainable=False)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': x_train_norm}, y=y_train, batch_size=10000, num_epochs=5, shuffle=True) 
regressor = tf.estimator.DNNRegressor(
    feature_columns=feature_columns, hidden_units=[1000, 250, 60, 15],
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0005, beta1=0.9, beta2=0.999, epsilon=0.1),
    batch_norm=True)
regressor.train(input_fn=train_input_fn)
print(datetime.now())

2018-09-25 22:41:30.577171
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp8kx1d9wm', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa4abff4710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmp8k

INFO:tensorflow:global_step/sec: 24.7558
INFO:tensorflow:loss = 223508.9, step = 7300 (4.039 sec)
INFO:tensorflow:global_step/sec: 23.7435
INFO:tensorflow:loss = 191632.56, step = 7400 (4.212 sec)
INFO:tensorflow:global_step/sec: 25.1637
INFO:tensorflow:loss = 175969.16, step = 7500 (3.974 sec)
INFO:tensorflow:global_step/sec: 24.1217
INFO:tensorflow:loss = 146146.88, step = 7600 (4.146 sec)
INFO:tensorflow:global_step/sec: 24.7296
INFO:tensorflow:loss = 218657.36, step = 7700 (4.044 sec)
INFO:tensorflow:global_step/sec: 24.1351
INFO:tensorflow:loss = 181600.16, step = 7800 (4.143 sec)
INFO:tensorflow:global_step/sec: 23.8271
INFO:tensorflow:loss = 169346.7, step = 7900 (4.197 sec)
INFO:tensorflow:global_step/sec: 23.477
INFO:tensorflow:loss = 193446.72, step = 8000 (4.260 sec)
INFO:tensorflow:global_step/sec: 25.3596
INFO:tensorflow:loss = 279751.3, step = 8100 (3.943 sec)
INFO:tensorflow:global_step/sec: 24.2232
INFO:tensorflow:loss = 173109.22, step = 8200 (4.128 sec)
INFO:tensorflo

INFO:tensorflow:global_step/sec: 24.8468
INFO:tensorflow:loss = 162873.97, step = 15500 (4.025 sec)
INFO:tensorflow:global_step/sec: 24.2881
INFO:tensorflow:loss = 206525.9, step = 15600 (4.117 sec)
INFO:tensorflow:global_step/sec: 24.4108
INFO:tensorflow:loss = 198334.92, step = 15700 (4.097 sec)
INFO:tensorflow:global_step/sec: 24.6578
INFO:tensorflow:loss = 178405.44, step = 15800 (4.056 sec)
INFO:tensorflow:global_step/sec: 24.9056
INFO:tensorflow:loss = 189532.03, step = 15900 (4.014 sec)
INFO:tensorflow:global_step/sec: 23.9883
INFO:tensorflow:loss = 171450.34, step = 16000 (4.169 sec)
INFO:tensorflow:global_step/sec: 23.9947
INFO:tensorflow:loss = 245684.53, step = 16100 (4.168 sec)
INFO:tensorflow:global_step/sec: 26.0254
INFO:tensorflow:loss = 198340.6, step = 16200 (3.842 sec)
INFO:tensorflow:global_step/sec: 26.3167
INFO:tensorflow:loss = 171718.47, step = 16300 (3.800 sec)
INFO:tensorflow:global_step/sec: 24.0668
INFO:tensorflow:loss = 184493.53, step = 16400 (4.155 sec)
IN

In [10]:
val_df = (pd
            .read_parquet("../../datasets/kaggle/new-york-city-taxi-fare-prediction/train_full_2")
            .sample(frac=0.20, random_state=420)
           )

In [11]:
print(val_df.shape)
print(val_df.columns)
print(val_df.info())

(10849366, 46)
Index(['fare_amount', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'distance_miles', 'is_pickup_JFK_new', 'is_dropoff_JFK_new',
       'is_pickup_EWR_new', 'is_dropoff_EWR_new', 'is_pickup_LGA_new',
       'is_dropoff_LGA_new', 'is_to_from_JFK_new', 'distance_to_center',
       'hour', 'weekday', 'year_2009', 'year_2010', 'year_2011', 'year_2012',
       'year_2013', 'year_2014', 'year_2015', 'hour_period_4', 'hour_period_0',
       'hour_period_1', 'hour_period_2', 'hour_period_5', 'hour_period_3',
       'distance_to_center_dropoff', 'pickup_zone_2', 'pickup_zone_0',
       'pickup_zone_1', 'pickup_zone_3', 'pickup_zone_4', 'pickup_zone_5',
       'pickup_zone_6', 'dropoff_zone_2', 'dropoff_zone_0', 'dropoff_zone_1',
       'dropoff_zone_3', 'dropoff_zone_4', 'dropoff_zone_5', 'dropoff_zone_6'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Int64Index: 10849366 entries,

In [12]:
x_val = val_df[cols].values
y_val = val_df[['fare_amount']].values

In [13]:
del val_df
gc.collect()

540

In [14]:
x_val_norm = scaler.transform(x_val)

In [15]:
del x_val
gc.collect()

0

In [16]:
val_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': x_val_norm}, y=y_val, num_epochs=1, shuffle=False)
scores = regressor.evaluate(input_fn=val_input_fn)
print('MSE (tensorflow): {0:f}'.format(scores['average_loss']))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-26-05:56:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8kx1d9wm/model.ckpt-21699
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-26-05:57:53
INFO:tensorflow:Saving dict for global step 21699: average_loss = 26.408344, global_step = 21699, label/mean = 11.322498, loss = 3380.255, prediction/mean = 11.31766
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 21699: /tmp/tmp8kx1d9wm/model.ckpt-21699
MSE (tensorflow): 26.408344


In [17]:
predictions = regressor.predict(input_fn=val_input_fn)
y_predicted = np.array(list(p['predictions'] for p in predictions))
y_predicted = y_predicted.reshape(np.array(y_val).shape)
score_sklearn = metrics.mean_squared_error(y_predicted, y_val)
print('MSE (sklearn): {0:f}'.format(score_sklearn))
y_predicted.shape

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8kx1d9wm/model.ckpt-21699
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
MSE (sklearn): 26.408339


(10849366, 1)

In [18]:
y_predicted[0:2]

array([[12.171405],
       [ 9.417773]], dtype=float32)

In [19]:
test_df = (pd
            .read_parquet("../../datasets/kaggle/new-york-city-taxi-fare-prediction/test_full_2")
           )

In [20]:
x_test = test_df[cols].values
x_test_norm = scaler.transform(x_test)

In [21]:
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': x_test_norm}, num_epochs=1, shuffle=False)
test_predictions = regressor.predict(input_fn=test_input_fn)
y_predicted = np.array(list(p['predictions'] for p in test_predictions))
y_predicted = y_predicted.reshape((9914,1))
#y_predicted = np.array(list(test_predictions))
#y_predicted = y_predicted.reshape((9914,1))
y_predicted.shape

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp8kx1d9wm/model.ckpt-21699
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


(9914, 1)

In [22]:
preds = [p[0] for p in y_predicted]
preds

[9.882126,
 8.886505,
 5.806313,
 8.584426,
 13.930721,
 10.556854,
 5.45793,
 49.569187,
 11.029599,
 6.320709,
 9.862189,
 14.405801,
 5.75776,
 7.7151785,
 7.949846,
 12.19937,
 6.814897,
 8.688636,
 5.9447103,
 6.641261,
 9.167123,
 8.594605,
 6.3643336,
 9.2197695,
 19.58703,
 6.801492,
 41.611088,
 23.258192,
 6.0909476,
 16.867273,
 11.651951,
 7.367543,
 6.266146,
 11.000851,
 9.808923,
 6.622689,
 6.856142,
 6.4094777,
 8.441293,
 6.0909476,
 18.423864,
 10.382085,
 20.180103,
 9.5543995,
 6.9120517,
 13.294905,
 6.7104964,
 8.780355,
 6.1811643,
 6.1364717,
 12.091449,
 12.474452,
 20.588472,
 5.7487016,
 7.560196,
 16.02978,
 13.294215,
 10.133587,
 13.822922,
 13.878989,
 9.351306,
 9.8363695,
 6.4409065,
 6.439627,
 7.185557,
 6.1759324,
 9.430144,
 6.266146,
 6.28748,
 18.34616,
 8.889829,
 7.425608,
 15.843623,
 7.192608,
 10.55511,
 11.047821,
 53.071617,
 7.203644,
 6.1759357,
 6.0909476,
 6.4683304,
 6.0909476,
 6.9120955,
 9.570887,
 20.102865,
 6.1758437,
 6.206925,

In [23]:
# Write the predictions to a CSV file which we can submit to the competition.
submission = pd.DataFrame(
    {'key': test_df.key, 'fare_amount': preds},
    columns = ['key', 'fare_amount'])
submission.to_csv('../../datasets/kaggle/new-york-city-taxi-fare-prediction/submission.csv', index = False)

In [24]:
submission.describe()

,fare_amount
count,9914.000000
mean,11.471549
std,8.633620
min,5.204337
25%,6.621692
50%,8.640927
75%,12.272739
max,112.193047
